<a href="https://colab.research.google.com/github/charlie9526/TaxiFare/blob/master/Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inports and Mounting

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Colab\ Notebooks/FAIR_TAXI


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from datetime import datetime
import random
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)              #data visualization library based on matplotlib
import numpy as np                  #support library for amtrices etc.
from scipy.stats import norm   
import warnings
warnings.filterwarnings('ignore')   #manage warnings
%matplotlib inline                  
#magic function in ipynb. ploting the graphs inline after the code
from math import sin, cos, sqrt, atan2, radians
from sklearn.neighbors import KNeighborsClassifier

from sklearn.svm import SVC,LinearSVC
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.cluster import KMeans
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
trip_id  = test_df['tripid']


# train_df = train_df[(train_df['additional_fare'].isna())&(train_df['label']=='correct')]

In [ ]:
################### ugly indexes
train_df = train_df[(train_df['tripid']!=213112837) | (train_df['tripid']!=193653017) | (train_df['tripid']!=209365576)]

## Definitions

In [ ]:
#elts write a lambda function
def get_duration(x):
  if x['pickup_time']==None or x['drop_time']==None:
    return None
  
  try :
    if int(x['pickup_time'].split("/")[1])>12 or int(x['drop_time'].split("/")[1])>12:
      gap = datetime.strptime(x['pickup_time'],"%m/%d/%Y %H:%M")-datetime.strptime(x['drop_time'],"%m/%d/%Y %H:%M")
    else:
      if int(x['pickup_time'].split("/")[1]) != int(x['drop_time'].split("/")[1]):
        gap = datetime.strptime(x['pickup_time'],"%m/%d/%Y %H:%M")-datetime.strptime(x['drop_time'],"%m/%d/%Y %H:%M")
      else:
        gap = datetime.strptime(x['pickup_time'],"%d/%m/%Y %H:%M")-datetime.strptime(x['drop_time'],"%d/%m/%Y %H:%M")
    # print (gap.total_seconds()/3600,x['pickup_time'] ,x['drop_time'])
  except :
    # print(int(x['pickup_time'].split("/")[0])>12 or int(x['drop_time'].split("/")[0])>12)
    return None
  # print(abs(gap.total_seconds()/3600), x['drop_time'],x['pickup_time'])
  disti =  abs(gap.total_seconds())
  if x['duration']>0:
    return x['duration']
  else:
    return disti

class MakeTripDurationFeature(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['time_gap'] = X.apply(lambda x:get_duration(x), axis=1)
        X = X.drop(labels=['pickup_time','drop_time'],axis=1)
        return X

# Change the label of binary values
class MakeTripDurationFeature(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        y = y.apply(lambda x:get_binary(x))
        # print (y);
        return self
    def transform(self,X):
        return X

# debugerr 
class debugerr(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        print (pd.DataFrame(X));
        return self
    def transform(self,X):
        return X

# lambda function for change label
def get_binary(x):
  if x==None:
    return 0
  if x == "correct":
    # print (x);
    return 1
  else:
    return 0

# set distance column
class set_distance(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X['distance'] = X.apply(lambda x:get_distance(x), axis=1)
      # print(X)
      # X = X.drop(labels=['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
      return X
# lambda column get distance
def get_distance(x):
  R = 6373.0
  lat1 = radians(float(x['drop_lat']))
  lon1 = radians(float(x['drop_lon']))
  lat2 = radians(float(x['pick_lat']))
  lon2 = radians(float(x['pick_lon']))

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c
  return distance

# remove columns
class remove_colums_unwanted(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X = X.drop(labels=['pick_lat','pick_lon','meter_waiting_fare','drop_lat','drop_lon','pickup_time','drop_time'],axis=1)
      return X


def encoding_label(label):
  if(label=='correct'):
    return 1
  else:
    return 0

def get_hour_value(x,date_time_column_name):
  if x[date_time_column_name]==None:
    return None
  return datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").hour

################### Lets get date out as a column
################### lambda function

def get_date_value(x,date_time_column_name):
  if x[date_time_column_name]==None:
    return None
  month = str(datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").month)
  date = str(datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").day)
  return int(month+date)

def calculate_direction(d_lon, d_lat):
    result = np.zeros(len(d_lon))
    l = np.sqrt(d_lon**2 + d_lat**2)
    result[d_lon>0] = (180/np.pi)*np.arcsin(d_lat[d_lon>0]/l[d_lon>0])
    idx = (d_lon<0) & (d_lat>0)
    result[idx] = 180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    idx = (d_lon<0) & (d_lat<0)
    result[idx] = -180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    return result
################### Here we have to add fare_per_hour column
################### write a lambda function

def get_fare_per_min(x):
  if x["fare"]>=0 and x["duration"]>0:
    minutes = x["duration"]/60.0
    val = x['fare']/minutes
    return val
  if x["duration"]==0:
    return 0
  return None

########### This is for getting fare per distance 
########### Lambda function
def get_fare_per_distance(x):
  if x['distance']==0:
    return 0
  if x['meter_waiting_fare']>0:
    return float(x['fare']-x['meter_waiting_fare'])/x['distance']
  else:
    return float(x['fare'])/x['distance']


## Pre-processing

check null values

In [ ]:
train_df['fare'].fillna(0,inplace=True)
test_df['fare'].fillna(0,inplace=True)

# print(train_df.describe())
# train_df = train_df[~( train_df['additional_fare'].isna()) | (train_df['label']!='correct')]
# print(train_df.describe())

train_df['additional_fare'].fillna(0,inplace=True)
test_df['additional_fare'].fillna(0,inplace=True)


brek cities

In [ ]:
# kmeans = KMeans(5, random_state=0)
# train_df['city'] = kmeans.fit(train_df[['drop_lon','drop_lat']]).predict(train_df[['drop_lon','drop_lat']])
# test_df['city'] = kmeans.fit(test_df[['drop_lon','drop_lat']]).predict(test_df[['drop_lon','drop_lat']])

encode label and create feature set


In [ ]:
train_df['label'] = train_df['label'].apply(encoding_label).values
features_df = train_df.loc[:, train_df.columns != 'label']
label_df = train_df['label']

remove unnecessary columns

In [ ]:
features_df = features_df.loc[:, features_df.columns != 'tripid']
test_df = test_df.loc[:, test_df.columns != 'tripid']

add some columns to dataset

In [ ]:

features_df['distance'] = features_df.apply(lambda x:get_distance(x), axis=1)
test_df['distance'] = test_df.apply(lambda x:get_distance(x), axis=1)


features_df['fare_per_min'] = features_df.apply(lambda x:get_fare_per_min(x),axis=1 )
test_df['fare_per_min'] = test_df.apply(lambda x:get_fare_per_min(x),axis=1 )


features_df['fare_per_distance'] = features_df.apply(lambda x:get_fare_per_distance(x),axis=1 )
test_df['fare_per_distance'] = test_df.apply(lambda x:get_fare_per_distance(x),axis=1 )

features_df['pickup_hour'] = features_df.apply(lambda x:get_hour_value(x,"pickup_time"),axis=1)
test_df['pickup_hour'] = test_df.apply(lambda x:get_hour_value(x,"pickup_time"),axis=1)

features_df = features_df.drop(labels=['drop_lat','drop_lon','pickup_time','pick_lat','pick_lon','drop_time'],axis=1)
test_df = test_df.drop(labels=['drop_lat','drop_lon','pickup_time','pick_lat','pick_lon','drop_time'],axis=1)

# features_df.to_csv("/content/drive/My Drive/Colab Notebooks/FAIR_TAXI/features_df.csv", index=True)


break others

In [ ]:
kmeans = KMeans(3, random_state=0)
ppp = ['fare_per_distance','fare']
ccc = 'fare_per_distance'
features_df[ccc] = kmeans.fit(features_df[ppp]).predict(features_df[ppp])
test_df[ccc] = kmeans.fit(test_df[ppp]).predict(test_df[ppp])

split train and test data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features_df, label_df, test_size=0.3, random_state=1)


## Model Pipelines

In [ ]:

numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='mean'))
])

numeric_cols = features_df.columns[features_df.dtypes != "object"].values

preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols)
    ],
    remainder = "drop"
)
print(numeric_cols)

['additional_fare' 'duration' 'meter_waiting' 'meter_waiting_fare'
 'meter_waiting_till_pickup' 'fare' 'distance' 'fare_per_min'
 'fare_per_distance' 'pickup_hour']


create random grid search cv for random forest

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor



# # Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 20)]
# # Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 15)]
max_depth.append(None)
# # Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20]
# # Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4 , 10]
# # Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid_random_forest = {
               'esti__n_estimators': n_estimators,
               'esti__max_features': max_features,
               'esti__max_depth': max_depth,
               'esti__min_samples_split': min_samples_split,
               'esti__min_samples_leaf': min_samples_leaf,
               'esti__bootstrap': bootstrap
               }
random_grid_svm = {
              "esti__C": np.arange(2, 10, 2),
              "esti__gamma": np.arange(0.1, 1, 0.2)
              }
random_grid_xgb = {
              'esti__gamma':[i/10.0 for i in range(0,5)],
              'esti__max_depth':range(3,10,2),
              'esti__min_child_weight':range(1,6,2)
}

wihout pca RF: 0.975462 (0.003489) with only 67 depth \
with pca RF: 0.974298 (0.003013)


In [ ]:

pca = PCA()
estimators = RandomForestClassifier(
      # bootstrap= False,
      random_state = 42,
      max_depth=67,
      # min_samples_split=5,
      # n_estimators = 500
)
remove_c = remove_colums_unwanted()

full_pipeline = Pipeline([
                          ("preprocessor", preprocessor),
                          # ('columnremove',remove_c),
                          # ("debuffe",debugerr()),
                          # ("pca",pca),
                          ("esti", estimators)
                          ])



In [ ]:
kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
cv_results = cross_val_score(
    full_pipeline, features_df, label_df, cv=kfold, scoring='f1')
print('%s: %f (%f)' % ("RF", cv_results.mean(), cv_results.std()))

RF: 0.974779 (0.003658)


In [ ]:
# rf_random_pipe = RandomizedSearchCV(full_pipeline, param_distributions = random_grid_svm, cv = 10, verbose=2, random_state=42, scoring ='f1')
# rf_random_pipe.fit(features_df,label_df)

In [ ]:
full_pipeline.fit(features_df,label_df)
predicted = full_pipeline.predict(test_df)
data_dict = { 'tripid':trip_id, 'prediction':predicted}
pd_test_result = pd.DataFrame(data=data_dict)


In [ ]:
pd_test_result[['tripid','prediction']].to_csv("/content/drive/My Drive/Colab Notebooks/FAIR_TAXI/result.csv", index=True)


In [ ]:
# rf_random_pipe.best_estimator_

In [ ]:
train_df.describe()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
count,1.717600e+04,17176.000000,16974.000000,16974.000000,16974.000000,16974.000000,17176.000000,17176.000000,17176.000000,17176.000000,17176.000000,17176.000000
mean,2.014549e+08,13.558299,1702.858077,629.074231,32.057666,112.466832,6.965499,79.998389,6.968587,79.997534,310.161649,0.902131
std,7.456830e+06,98.402411,8120.160818,8063.260669,465.483573,442.054812,0.210760,0.243384,0.378669,0.359648,603.522765,0.297146
min,1.891236e+08,0.000000,0.000000,0.000000,0.000000,0.000000,5.937600,79.819100,5.941860,45.307700,0.000000,0.000000
25%,1.947619e+08,10.500000,548.000000,33.000000,0.000000,26.000000,6.865077,79.870400,6.865625,79.870100,123.427500,1.000000
50%,2.013582e+08,10.500000,1009.500000,123.000000,0.116600,73.000000,6.914105,79.898600,6.916160,79.899100,209.540000,1.000000
75%,2.083081e+08,10.500000,1803.000000,351.000000,14.341800,144.000000,7.043057,79.956600,7.047195,79.960225,364.315000,1.000000
max,2.138173e+08,12795.030000,453872.000000,453650.000000,26198.601400,51288.000000,8.051010,81.009600,48.132000,81.009700,26945.990000,1.000000
